In [5]:
#Learning Neural Networks
import random
import math
import csv

In [6]:
#A few helper functions representing a common activation function and its derivative
def sigmoid(x):
    return 1/(1 + math.exp(-x))

def sigmoid_derivative(s):
    return s * (1.0 - s)

In [7]:
class Neuron(object):
    def __init__(self, input_count, activation_func=sigmoid):
        self.bias = random.random()
        self.weights = [random.random() for i in range(input_count)]
        self.input_count = input_count
        self.activation_func = activation_func
    
    #Takes the weighted sum of inputs then applies a activation function 
    #to narmalize the sum
    def dot_product(self, inputs):
        """if len(inputs) != self.input_count:
            pass #raise error"""
        weighted_input_sum = self.bias + sum([(self.weights[i] * inputs[i]) for i in range(len(inputs))])
        self.output = self.activation_func(weighted_input_sum)
        return self.output
            
def makeNeuronLayer(input_count, neuron_count):
    return [Neuron(input_count) for i in range(neuron_count)]
        

In [52]:
def one_hot_encode(val, max_val):
    encoded_val = [0 for i in range(max_val)]
    encoded_val[val-1] = 1
    return encoded_val

#TODO: Add support for regression later
class NeuralNetwork(object):
    def __init__(self, n_inputs, n_hidden_neuron_count, n_outputs, n_hidden_layers, learning_rate=0.2, debug=False):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.learning_rate = learning_rate
        self.debug = debug
        
        self.layers = []
        #input -> hidden
        self.layers.append(makeNeuronLayer(n_inputs, n_hidden_neuron_count))
        #hidden -> hidden
        for i in range(n_hidden_layers):
            self.layers.append(makeNeuronLayer(n_hidden_neuron_count, n_hidden_neuron_count))
        #hidden -> output
        self.layers.append(makeNeuronLayer(n_hidden_neuron_count, n_outputs))
        
    def backprob(self, target):
        """target = one_hot_encode(target, self.n_outputs)
        
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            errors = []
            if i != len(self.layers) - 1:
                for j in range(len(layer)):
                    neuron_error = 0.0
                    for neuron in self.layers[i + 1]:
                        neuron_error += (neuron.weights[j] * neuron.delta)
                    errors.append(neuron_error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(target[j] - neuron.output)
            
            for j in range(len(layer)):
                neuron = layer[j]
                neuron.delta = errors[j] * sigmoid_derivative(neuron.output)"""
        target = one_hot_encode(target, self.n_outputs)
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            errors = []
            if i != (len(self.layers) - 1):
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in self.layers[i + 1]:
                        error += (neuron.weights[j] * neuron.delta)
                    errors.append(error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    #error = 0.5 * (target[j] - neuron.output)**2
                    error = neuron.output - target[j]
                    """print("STEP")
                    print("\ttarget:", target[j])
                    print("\toutput:", neuron.output)
                    print("\terror:", error)"""
                    errors.append(error)
                    
            for j in range(len(layer)):
                neuron = layer[j]
                neuron.delta = errors[j] * sigmoid_derivative(neuron.output)

    def update_weights(self, inputs):
        for i in range(len(self.layers)):
            if i != 0:
                inputs = [neuron.output for neuron in self.layers[i - 1]]
                
            for neuron in self.layers[i]:
                for j in range(len(inputs)):
                    neuron.weights[j] += self.learning_rate * neuron.delta * inputs[j]
                neuron.bias += self.learning_rate * neuron.delta

    def train(self, rows, n_epoch=1):
        """for epoch in range(n_epoch):
            sum_error = 0.0
            for row in rows:
                inputs = row[0:-1]
                target = row[-1]

                output = self.predict(inputs)
                sum_error += (target - output)**2
                
                self.backprob(output)
                self.update_weights(inputs)
            print(">Epoch: %d %.3f" % (epoch, sum_error))
            
        #Accuracy test, after training
        corr_count = 0
        for row in rows:
            inputs = row[0:-1]
            target = row[-1]
            output = self.predict(inputs)
            if output is target:
                corr_count += 1
            print(output, "--", target)
        print("Accuracy:", corr_count/float(len(rows)))"""
        for epoch in range(n_epoch):
            sum_error = 0
            for row in rows:
                inputs = row[0:-1]
                target = row[-1]
                output = self.predict(inputs)
                
                sum_error += 0.5 * (output - target)**2
                #sum_error = (target-output)
                
                self.backprob(target)
                self.update_weights(inputs)
            print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, self.learning_rate, sum_error))

    def predict(self, inputs):
        for layer in self.layers:
            inputs = [neuron.dot_product(inputs) for neuron in layer]
        return inputs.index(max(inputs)) #This forces classification, works great with one-hot encoded data

In [53]:
import csv
#Helper method for getting all the rows of a .csv
def read_csv_data(filename, skip_first=False):
    lines = []
    with open(filename, newline='') as file:
        reader = csv.reader(file)
        if skip_first: #Skips the first row, possibly due to header
            next(reader, None)
        lines = [row for row in reader]
    return lines

#Attempt to converts the given columns of a list to floats
def string_columns_to_floats(rows, column_ls):
    for ls in rows:
        for i in column_ls:
            try:
                ls[i] = float(ls[i])
            except ValueError:
                pass
            
#takes columns that are classification represented as string and converts them to a number classifications
def number_encode_classification(rows, column_ls):
    for column in column_ls:
        assoc_values = {}
        value_count = 0

        for row in rows:
            if row[column] not in assoc_values:
                assoc_values[row[column]] = value_count
                value_count += 1
            row[column] = assoc_values[row[column]]
            
#Moves all list items into random positions
def shuffle_list_items(rows):
    return random.sample(rows, len(rows))

data = read_csv_data("datasets/iris.csv", skip_first=True)

string_columns_to_floats(data, [0,1,2,3])
number_encode_classification(data, [4])
data = shuffle_list_items(data)

In [54]:
#Some tests that fill the neural network with known good data to ensure that proper outputs occur
n1 = NeuralNetwork(4, 5, 3, 2)
print(n1.layers[1][0].weights[0])
print(n1.layers[1][0].weights[1])
print(n1.layers[1][0].weights[2])
n1.train(data, n_epoch=50)
print(n1.layers[1][0].weights[0])
print(n1.layers[1][0].weights[1])
print(n1.layers[1][0].weights[2])

0.5620888731389184
0.3551047099196044
0.058944465291167925
>epoch=0, lrate=0.200, error=56.000
>epoch=1, lrate=0.200, error=53.000
>epoch=2, lrate=0.200, error=53.500
>epoch=3, lrate=0.200, error=52.000
>epoch=4, lrate=0.200, error=52.000
>epoch=5, lrate=0.200, error=50.000
>epoch=6, lrate=0.200, error=50.000
>epoch=7, lrate=0.200, error=50.000
>epoch=8, lrate=0.200, error=50.000
>epoch=9, lrate=0.200, error=50.000
>epoch=10, lrate=0.200, error=50.000
>epoch=11, lrate=0.200, error=50.000
>epoch=12, lrate=0.200, error=50.000
>epoch=13, lrate=0.200, error=50.000
>epoch=14, lrate=0.200, error=50.000
>epoch=15, lrate=0.200, error=50.000
>epoch=16, lrate=0.200, error=50.000
>epoch=17, lrate=0.200, error=50.000
>epoch=18, lrate=0.200, error=50.000
>epoch=19, lrate=0.200, error=50.000
>epoch=20, lrate=0.200, error=50.000
>epoch=21, lrate=0.200, error=50.000
>epoch=22, lrate=0.200, error=50.000
>epoch=23, lrate=0.200, error=50.000
>epoch=24, lrate=0.200, error=50.000
>epoch=25, lrate=0.200, er

In [63]:
#Example link - https://matthewmazur.files.wordpress.com/2018/03/neural_network-9.png
#https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
n2 = NeuralNetwork(2, 2, 2, 1, debug=True)
n2.layers[0][0].weights = [.15, .20]
n2.layers[0][1].weights = [.25, .30]
n2.layers[1][0].weights = [.40, .45]
n2.layers[1][1].weights = [.50, .55]
n2.layers[0][0].bias = 0.35
n2.layers[0][1].bias = 0.35
n2.layers[1][0].bias = 0.60
n2.layers[1][1].bias = 0.60

n2.predict([0.05, 0.10])
print(n2.layers[1][0].output, "==", 0.751)
print(n2.layers[1][1].output, "==", 0.772)

print(sigmoid_derivative(n2.layers[1][0].output), "==", 0.1868)

n2.backprob(1)
output_layer = n2.layers[1]
print(output_layer[0].weights)
print(output_layer[1].weights)
#Checks out

0.7513650695523157 == 0.751
0.7729284653214625 == 0.772
0.18681560180895948 == 0.1868
[0.4, 0.45]
[0.5, 0.55]
